In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [11]:
dataset = tfds.load('imdb_reviews',as_supervised=True)

In [12]:
train_dataset,test_dataset=dataset['train'],dataset['test']
batch_size=32
train_dataset=train_dataset.shuffle(10000).batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

In [13]:
example,label=next(iter(train_dataset))
print('Text:\n',example.numpy()[0])
print('\nLabel: ', label.numpy()[0])

Text:
 b"Oh dear. I was so disappointed that this movie was just a rip-off of Japan's Ringu. Well, I guess the U.S. made their version of it as well, but at least it was an outright remake. So, so sad. I very much enjoy watching Filipino movies and know some great things can come out of such a little country, so I can't believe this had to happen. Claudine and Kris are such big names there, surprised they would be affiliated with plagiarism. To any aspiring movie makers out there in the Philippines: You do not have to stoop this low to make money. There are many movie buffs that are watching the movies Filipinos put out and enjoying them!"

Label:  0


text visualization 